In [ ]:
Персонализация
film id for calculations
96

In [1]:
!hdfs dfs -cat /labs/laba01/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.item

-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item


In [2]:
from  pyspark.sql import SparkSession

In [13]:
spark = SparkSession.Builder().master('local[1]').appName('spark_by_art').getOrCreate()
print('done!')

done!


In [8]:
from pyspark.context import SparkContext

In [14]:
sc = SparkContext('local','tst1')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark_by_art, master=local[1]) created by getOrCreate at <ipython-input-3-7a517be01f7f>:1 

In [15]:
sc = spark.SparkContext

AttributeError: 'SparkSession' object has no attribute 'SparkContext'

In [63]:
spark.stop()

In [3]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


In [4]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "may RDD app")

sc = SparkContext(conf=config)

In [64]:
sc.stop()

In [49]:
!hdfs dfs -ls "/labs/laba01/ml-100k/u.item"

-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item


In [58]:
!hdfs dfs -cp "/labs/laba01/ml-100k/u.item" "/data/home/artem.kalmykov/LAB_01/labs/laba01/ml-100k/u.item"

cp: `/data/home/artem.kalmykov/LAB_01/labs/laba01/ml-100k/u.item': No such file or directory: `hdfs://spark-master-1.newprolab.com:8020/data/home/artem.kalmykov/LAB_01/labs/laba01/ml-100k/u.item'


In [6]:
rdd_film = sc.textFile("/labs/laba01/ml-100k/u.item")

In [37]:
type(rdd_film)

pyspark.rdd.RDD

In [7]:
rdd_film.count()

1682

In [9]:
rdd_film.take(10)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0',
 '6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0',
 '8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title

In [47]:
(rdd_film.map(lambda x: (x.split("|")[0:3]))
         .filter(lambda x: "96"  in x)
         
).take(5)

[['96', 'Terminator 2: Judgment Day (1991)', '01-Jan-1991']]

In [ ]:
f.lit()

In [ ]:
/labs/laba01/ml-100k/u.data

In [5]:
rdd_data = sc.textFile( "/labs/laba01/ml-100k/u.data")

In [15]:
rdd_data.take(2)

['196\t242\t3\t881250949', '186\t302\t3\t891717742']

In [6]:
rdd_data_my = (rdd_data
               .map(lambda x: (x.split("\t")[1:3]))
                .filter(lambda x: '96' == x[0] )
                .map(lambda x: [x[1], 1] )
                .reduceByKey( (lambda x,  y : x + y) )
                
         
              )

# rdd_data_my.count() #295
rdd_data_my.take(10)

[('1', 6), ('4', 123), ('5', 103), ('3', 43), ('2', 20)]

In [110]:
123+6+103+43+20

295

In [7]:
rdd_data_all = (rdd_data
               .map(lambda x: (x.split("\t")[1:3]))
#                 .filter(lambda x: '96' == x[0] )
                .map(lambda x: [x[1], 1] )
                .reduceByKey( (lambda x,  y : x + y) )
                
         
              )

# rdd_data_my.count() #295
rdd_data_all.take(10)

[('1', 6110), ('4', 34174), ('3', 27145), ('2', 11370), ('5', 21201)]

In [36]:
rdd_data_my_s = rdd_data_my.sortByKey().map(lambda x: x[1] ).take(6)
rdd_data_my_s

[6, 20, 43, 123, 103]

In [39]:
rdd_data_all_s = rdd_data_all.sortByKey().map(lambda x: x[1] ).take(6)
rdd_data_all_s

[6110, 11370, 27145, 34174, 21201]

In [40]:
result_dict = {'hist_film':rdd_data_my_s , 'hist_all': rdd_data_all_s   }
result_dict

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [41]:
type(result_dict)

dict

In [43]:
import json as jj

In [45]:
with open('lab01.json', 'w') as fp:
    jj.dump(result_dict, fp)
    print('done')

done


In [46]:
sc.stop()
print('stoped')

stoped
